# 用特征分解做图的连通性分析

## 介绍

用**特征分解（谱分解）做图的连通性分析**，通常属于**谱图论（Spectral Graph Theory）**的内容，核心思想是：
**通过图相关矩阵的特征值和特征向量，判断图是否连通、连通分量个数，以及节点之间的结构关系。**

下面按逻辑一步一步说明。

---

一、图的表示方式

设图 $G=(V,E)$，有 $n$ 个节点。

1）邻接矩阵
$A \in \mathbb{R}^{n\times n}$，
$$
A_{ij} =
\begin{cases}
1, & (i,j)\in E \\
0, & \text{otherwise}
\end{cases}
$$

2）度矩阵
(D) 是对角矩阵，
$$
D_{ii} = \sum_j A_{ij}
$$

3）拉普拉斯矩阵（最常用）

* 非归一化拉普拉斯：
  $$
  L = D - A
  $$
* 归一化拉普拉斯：
  $$
  L_{\text{sym}} = I - D^{-1/2} A D^{-1/2}
  $$

**连通性分析几乎总是基于拉普拉斯矩阵。**

---

二、核心定理（判断连通性的关键）

1）零特征值与连通分量

对于非归一化拉普拉斯矩阵 $L$：

* $L$ 至少有一个特征值为 0
* **特征值 0 的重数 = 图的连通分量个数**

因此：

* 只有一个 0 特征值 → 图是连通的
* 有 $k$ 个 0 特征值 → 图有 $k$ 个连通分量

这是连通性分析最直接、最重要的结论。

---

三、代数连通度（连通“强度”）

设拉普拉斯特征值排序为：
$$
0 = \lambda_1 \le \lambda_2 \le \cdots \le \lambda_n
$$

* $\lambda_2$ 称为 **代数连通度（Algebraic Connectivity）**
* 若 $\lambda_2 > 0$，图连通
* $\lambda_2$ 越大，图“越连通”
* $\lambda_2$ 趋近 0，图接近被“割裂”

这在网络鲁棒性、聚类、社区发现中非常重要。

---

四、特征向量与连通分量结构

1）零特征值对应的特征向量

当图有多个连通分量时：

* 每个连通分量对应一个零特征向量
* 特征向量在该分量节点上为常数，在其他位置为 0

这些向量张成的空间可以直接用于识别连通分量。

2）Fiedler 向量（第二小特征值对应向量）

* $\lambda_2$ 对应的特征向量称为 **Fiedler 向量**
* 其符号或数值大小可用于：

  * 图二分
  * 谱聚类
  * 社区划分

---

五、用特征分解做连通性分析的基本流程

1. 构建图的邻接矩阵 $A$
2. 计算度矩阵 $D$
3. 构造拉普拉斯矩阵 $L=D-A$
4. 做特征分解：
  $$
  L = U \Lambda U^\top
  $$
5. 分析：
    * 0 特征值个数 → 连通分量数
    * $\lambda_2$ → 连通强度
    * 对应特征向量 → 连通结构与聚类

---

六、一个直观例子

如果图由两个完全不相连的子图组成：

* 拉普拉斯矩阵会是块对角形式
* 特征值中会出现两个 0
* 特征向量在不同块上分离

这在谱空间中表现为明显的“分裂结构”。

---

七、总结一句话

**图的连通性可以通过拉普拉斯矩阵的特征分解来刻画：
零特征值的重数决定连通分量个数，第二小特征值刻画连通强度，特征向量揭示连通结构。**


## 实例

### 第一步：导入库

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

### 第二步：构造一个“部分不连通”的图

我们手工构造一个图：

- 节点 0–4 是一个连通分量
- 节点 5–8 是另一个连通分量

In [ ]:
G = nx.Graph()

edges = [
    (0,1),(1,2),(2,3),(3,4),(4,0),(1,3),   # 第一个连通分量
    (5,6),(6,7),(7,8),(8,5)                # 第二个连通分量
]

G.add_edges_from(edges)


### 第三步：可视化原始图结构

In [ ]:
plt.figure(figsize=(6,4))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_size=600)
plt.title("Original Graph")
plt.show()


你应该能明显看到两个互不相连的子图。

### 第四步：构造邻接矩阵、度矩阵、拉普拉斯矩阵

In [ ]:
A = nx.to_numpy_array(G)
D = np.diag(A.sum(axis=1))
L = D - A

print("Adjacency matrix A:\n", A)
print("\nDegree matrix D:\n", D)
print("\nLaplacian matrix L:\n", L)

### 第五步：拉普拉斯矩阵特征分解

In [ ]:
eigvals, eigvecs = np.linalg.eigh(L)

print("Eigenvalues:\n", eigvals)

这里你会看到：

- 有两个非常接近 0 的特征值
- 说明图有 2 个连通分量

### 第六步：特征值可视化（连通性一眼就能看出来）

In [ ]:
plt.figure(figsize=(6,4))
plt.stem(eigvals)
plt.xlabel("Index")
plt.ylabel("Eigenvalue")
plt.title("Eigenvalues of Graph Laplacian")
plt.show()

经验判断：

- 0 特征值的个数 = 连通分量数
- 第二小特征值 $\lambda_2$ 衡量连通强度

### 第七步：零特征值对应的特征向量

In [ ]:
eigvecs_zero = eigvecs[:, :2]  # 前两个接近0的特征值
print(eigvecs_zero)

这个输出**非常标准、而且“教科书级正确”**，你这一步其实已经把**连通分量的谱结构**完整算出来了。下面我不跳步，按“怎么看 → 为什么这样 → 能得出什么结论”来解读。

---

先把矩阵写成“按节点看”的形式

你有 9 个节点（0–8），`eigvecs_zero` 是一个 (9 \times 2) 矩阵：

```
节点 i   第1列        第2列
0      -0.0         0.4472136
1      -0.0         0.4472136
2      -0.0         0.4472136
3      -0.0         0.4472136
4      -0.0         0.4472136
5       0.5         0.0
6       0.5         0.0
7       0.5         0.0
8       0.5         0.0
```

每一行对应一个节点。

---

一、最核心的事实（结论先行）

- 图 **有两个连通分量**
- 每一个 0 特征值 → 一个连通分量
- 每一列特征向量 → 某个连通分量的“指示函数”

你看到的这两个特征向量，正好“一人管一个分量”。

---

二、为什么会出现这种“分块常数”的结构

拉普拉斯矩阵有一个非常重要的性质：

> 在同一个连通分量内，
> 零特征值对应的特征向量 **必须是常数**

原因直觉上是：

* 拉普拉斯刻画的是“相邻节点差异”
* 零特征值意味着“没有差异”
* 只能是“同一分量内全相等”

不同连通分量之间：

* 完全没有边
* 特征向量可以彼此独立取值

---

三、逐列解读你现在看到的结果

第一列（第一个 0 特征向量）：

```
节点 0–4 : 0
节点 5–8 : 0.5
```

含义：

* 它“只在第二个连通分量上有值”
* 在第一个连通分量上为 0
* 是**第二个连通分量的指示向量**

第二列（第二个 0 特征向量）：

```
节点 0–4 : 0.4472136
节点 5–8 : 0
```

含义：

* 它“只在第一个连通分量上有值”
* 是**第一个连通分量的指示向量**

---

四、为什么是 0.4472136 和 0.5，而不是 1

这是一个**非常好的细节问题**。

这些特征向量是 **单位正交归一化的**：

$$
|\mathbf{v}|_2 = 1
$$

第一个连通分量有 5 个节点：

$$
\sqrt{5 \times c^2} = 1 \Rightarrow c = \frac{1}{\sqrt{5}} \approx 0.4472136
$$

第二个连通分量有 4 个节点：

$$
\sqrt{4 \times c^2} = 1 \Rightarrow c = \frac{1}{2} = 0.5
$$

所以数值正好对应**连通分量大小**。

---

五、为什么有 -0.0

这是浮点数数值误差 + 符号约定导致的：

* `-0.0` 和 `0.0` 完全等价
* 特征向量整体乘以 `-1` 也是合法的
* 不影响任何物理 / 图论含义

可以直接当成 0 看。

---

六、能直接用它做什么（非常实用）

1️⃣ 自动找连通分量（不用 networkx）

```python
labels = np.argmax(np.abs(eigvecs_zero), axis=1)
print(labels)
```

输出会是：

```
[1 1 1 1 1 0 0 0 0]
```

这就是节点所属的连通分量编号。

---

2️⃣ 用来做谱聚类的“理想输入”

零特征向量张成的子空间，本质上就是：

* 连通分量的低维嵌入
* 理想情况下可以被完全线性分离

---

七、一句话总结你这个结果

你的 `eigvecs_zero` 精确地编码了图的连通结构：
- 每一列是一个连通分量的指示向量，
- 常数值反映分量大小，
- 零特征值的个数等于连通分量个数。

如果你愿意，下一步我可以给你演示：

👉 **加一条跨分量的边后，这两列会如何“塌缩成一列”**，这对理解代数连通度特别有冲击力。


In [ ]:
labels = np.argmax(np.abs(eigvecs_zero), axis=1)
print(labels)

### 第八步：用特征向量给节点“着色”

In [ ]:
plt.figure(figsize=(6,4))

colors = eigvecs_zero[:, 1]  # 任选一个零特征向量
nx.draw(G, pos,
        node_color=colors,
        cmap="coolwarm",
        with_labels=True,
        node_size=600)

plt.title("Connected Components Revealed by Eigenvector")
plt.colorbar(plt.cm.ScalarMappable(cmap="coolwarm"), ax=plt.gca())
plt.show()


这一步非常关键：谱空间中，连通分量自然分开了

### 第九步：Fiedler 向量（第二小非零特征值）

In [ ]:
fiedler_vector = eigvecs[:, 2]  # 第三个特征值（λ₂）

plt.figure(figsize=(6,4))
nx.draw(G, pos,
        node_color=fiedler_vector,
        cmap="viridis",
        with_labels=True,
        node_size=600)

plt.title("Fiedler Vector Visualization")
plt.colorbar(plt.cm.ScalarMappable(cmap="viridis"), ax=plt.gca())

plt.show()

Fiedler 向量在：

- 谱聚类
- 图划分
- 社区发现

中是核心工具。

### 第十步：在谱空间中画节点分布

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(eigvecs[:,0], eigvecs[:,1], s=80)

for i in range(len(G.nodes)):
    plt.text(eigvecs[i,0], eigvecs[i,1], str(i))

plt.xlabel("1st eigenvector")
plt.ylabel("2nd eigenvector")
plt.title("Nodes in Spectral Space")
plt.axhline(0, color='gray', linestyle='--')
plt.axvline(0, color='gray', linestyle='--')
plt.show()

你会看到：

- 节点在谱空间中自动聚成两团

这正是谱方法的几何直觉

完整展示了：

- 拉普拉斯特征值 → 连通分量个数
- 零特征向量 → 连通分量结构
- Fiedler 向量 → 图划分方向
- 谱空间 → 连通性几何表达

## 加一列

第一步：在原图上加一条跨分量的边

比如把节点 4 和节点 5 连起来（刚好是两个分量的边界）：

In [ ]:
G2 = G.copy()
G2.add_edge(4, 5)

第二步：重新画图（你会看到已经连成一块了）

In [ ]:
plt.figure(figsize=(6,4))
pos2 = nx.spring_layout(G2, seed=42)
nx.draw(G2, pos2, with_labels=True, node_size=600)
plt.title("Graph after adding a bridge edge")
plt.show()


第三步：重新构造拉普拉斯并做特征分解

In [ ]:
A2 = nx.to_numpy_array(G2)
D2 = np.diag(A2.sum(axis=1))
L2 = D2 - A2

eigvals2, eigvecs2 = np.linalg.eigh(L2)

print("Eigenvalues:\n", eigvals2)


注意：

- 只剩一个 0
- 第二小特征值现在是一个很小的正数

第四步：对比“塌缩前 vs 塌缩后”的零空间

之前（两个分量）：

    零空间维数 = 2
    [ 分量1指示向量 , 分量2指示向量 ]

现在（连通）：

    零空间维数 = 1
    [ 全1向量 ]

In [ ]:
print(eigvecs2[:, 0])

你会看到它在所有节点上几乎是常数（归一化后的）。

 第五步：原来的“第二列”去了哪里？

关键来了。

看现在的 Fiedler 向量：

In [ ]:
fiedler = eigvecs2[:, 1]

fig, ax = plt.subplots(figsize=(6,4))
nx.draw(G2, pos2,
        node_color=fiedler,
        cmap="coolwarm",
        with_labels=True,
        node_size=600,
        ax=ax)

sm = plt.cm.ScalarMappable(
    cmap="coolwarm",
    norm=plt.Normalize(vmin=fiedler.min(), vmax=fiedler.max())
)
sm.set_array([])
plt.colorbar(sm, ax=ax)

ax.set_title("Fiedler vector after adding bridge edge")
plt.show()


第六步：用一句话解释“塌缩成一列”到底是什么意思

**加边前：**

* 两个分量完全独立
* 可以各自“随便取常数”
* 零空间是 2 维

**加边后：**

* 两个分量被“约束在一起”
* 只能整体一起取常数
* 零空间退化成 1 维
* 原来的“分量差异方向”被迫挤到了下一个特征值里

这个“被挤出来的方向”，
**正是 Fiedler 向量**。


第七步：一个非常形象的物理直觉

把图想成弹簧系统：

* 节点 = 质点
* 边 = 弹簧

加边前：

* 两个系统彼此独立
* 可以各自整体平移（零能量模式 ×2）

加边后：

* 用一根软弹簧连起来
* 只能整体一起平移（零模 ×1）
* 相对运动 → 需要一点能量 $(\lambda_2 > 0)$


最后一句总结

**“塌缩成一列”不是消失，而是：
零特征向量的自由度被约束，
分量差异从零特征值上升为 Fiedler 模式。**